In [ ]:
#hide
#from your_lib.core import *

# `deepfeatx`: Deep Learning Feature Extractor of Images from Transfer Learning Models

> Helper for automatic extraction of features from images (and soon text as well) from transfer learning models like ResNet, VGG16 and EfficientNet.

## Install

`pip install deepfeatx`